In [2]:
import sqlite3
import pandas as pd
import os
import random
from IPython.display import HTML


In [3]:
folder = "document_imgs"

In [4]:
files = next(os.walk(folder))

In [5]:
files =[os.path.join(folder,f) for f in files[2]]

In [7]:
len(files)

197

In [36]:
import pandas as pd

In [39]:
with sqlite3.connect(files[0]) as con:
    df = pd.read_sql("select * from sqlite_master", con)
df.style

,type,name,tbl_name,rootpage,sql
0,table,img,img,2,"CREATE TABLE img (urn int, filename text, tag_name text, elem_ID text, elem_type text, elem_height int, elem_width int, elem_hpos int, elem_vpos int, word_num int, para_num int, page_num int, img_num int, PRIMARY KEY (urn, img_num)) WITHOUT rowid"
1,index,_et_,img,419211,"CREATE INDEX _et_ on img (elem_type, filename)"


In [42]:
with sqlite3.connect(files[0]) as con:
    df = pd.read_sql("select * from img limit 4", con)
df.style

,urn,filename,tag_name,elem_ID,elem_type,elem_height,elem_width,elem_hpos,elem_vpos,word_num,para_num,page_num,img_num
0,203950000,tidenskrav_null_null_19560611_47_131_1-1_001_null.xml,ComposedBlock,P1_CB00001,Illustration,1044,1155,170,2334,708,33,1,1
1,203950000,tidenskrav_null_null_19560611_47_131_1-1_001_null.xml,ComposedBlock,P1_CB00002,Illustration,753,1675,796,4317,1314,46,1,2
2,203950000,tidenskrav_null_null_19560611_47_131_1-1_001_null.xml,ComposedBlock,P1_CB00003,Illustration,303,979,2581,1328,1920,68,1,3
3,203950000,tidenskrav_null_null_19560611_47_131_1-1_003_null.xml,ComposedBlock,P3_CB00001,Illustration,520,567,137,808,7110,208,3,4


In [44]:
urls = df2url_type(df)

In [48]:
urn2url(*urls[0][0])

'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_tidenskrav_null_null_19560611_47_131_1-1_001_null/268,3687,1824,1649/full/0/native.jpg'

In [12]:
small_scale = 0.59
large_scale = 1.58

In [13]:
def df2url_type(df):
    if df.filename[0].startswith("digibok"):
        picture_data = [(
            (
                f"URN:NBN:no-nb_{r.filename.split('.')[0]}", 
                r.elem_hpos, 
                r.elem_vpos, 
                r.elem_width, 
                r.elem_height
            ), 
            r.elem_type
        ) for _,r in df["filename elem_height elem_width elem_hpos elem_vpos elem_type".split()].iterrows()]
    else:
        picture_data = [((f"URN:NBN:no-nb_digavis_{r.filename.split('.')[0]}", r.elem_hpos, r.elem_vpos, r.elem_width, r.elem_height), r.elem_type) for _,r in df["filename elem_height elem_width elem_hpos elem_vpos elem_type".split()].iterrows()]
    return picture_data

In [14]:
def urn2url(urn, hpos, vpos, width, height, scale = large_scale, part = None):
    if part is None:
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(width * scale), 
            height = int(height * scale), 
            vpos = int(vpos * scale), 
            hpos = int(hpos * scale)
        )
    else:
        url = f"https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg"
    return url

In [15]:
def display_finds_df_type(df, cols = 5, scale=large_scale, size = 200):
    """A list of urls in r is displayed as HTML"""
    data = df2url_type(df)
    
    quotient, remainder = divmod(len(data), cols)
    j = 0
    rows = ""
    for i in range(quotient):
        cells = ''.join([f"<td><strong>{row[1]}</strong></td><td><img src='{urn2url(*row[0], scale=scale)}' width={size}</td>" for row in data[j:j+cols]])
        j += cols
        rows += f"<tr>{cells}</tr>"
    cells = ''.join([f"<td><strong>{row[1]}</strong></td><td><img src='{urn2url(*row[0], scale=scale)}' width={size}</td>" for row in data[j:j+remainder]])
    rows += f"<tr>{cells}</tr>"
    return HTML(f"""<html><head></head>
     <body>
     <table>
     {rows}
     </table>
     </body>
     </html>""")

## Read images

In [31]:
%%time
with sqlite3.connect(random.choice(files)) as con:
    df = pd.read_sql("select * from img where elem_type='' limit 200", con)

CPU times: user 6.02 ms, sys: 0 ns, total: 6.02 ms
Wall time: 11.7 ms


In [33]:
display_finds_df_type(df, scale = small_scale, size=300)

,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,10339,1449,653/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,10992,1449,1582/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,10339,1449,2236/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,7932,1449,758/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,8690,1449,1648/full/0/native.jpg' width=300"
,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,7932,1449,2406/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,5800,1449,559/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,6359,1449,1572/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/4291,5800,1449,2132/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/5741,5913,3324,360/full/0/native.jpg' width=300"
,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/5741,6273,3324,322/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/5741,6595,1412,1231/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/7153,6595,1912,1231/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/5741,5913,3324,1913/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,7629,1412,378/full/0/native.jpg' width=300"
,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,8008,1412,1374/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,7629,1412,1752/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,9382,1412,236/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,9618,1412,900/full/0/native.jpg' width=300",,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digavis_hamarstiftstidendenittentjueni_null_null_19570927_111_223_1-1_001_null/2879,9382,1412,1136/full/0/native.jpg' width=300"
,"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-

In [26]:


def counts(f):
    fname = f.split('/')[1].split('.')[0]
    with sqlite3.connect(f) as con:
        res = pd.read_sql(f"select elem_type, count(*) as {fname} from img group by elem_type", con)
    return res

def index(f):
    with sqlite3.connect(f) as con:
        con.execute("create index _et_ on img (elem_type, filename)")

In [17]:

f = random.choice(files)
a = counts(f)

In [18]:
a

,elem_type,img_201050000_201074999
0,,4835161
1,Advertisement,290700
2,Cartoon,43167
3,Graph,1188
4,Illustration,508009
5,Map,1830
6,Photo,199942
7,Table,7705


for f in files:
    index(f)

In [27]:
all_files = {}
for f in files:
    try:
        fname = f.split('/')[1].split('.')[0]
        all_files[fname] = counts(f)
    except:
        print(fname)

img_202325000_202349999


In [28]:
tot = pd.concat([all_files[f].set_index('elem_type') for f in all_files], axis = 1)

In [101]:
pd.DataFrame(tot.fillna(0).sum(axis=1), columns=["antall"]).sort_values(by="antall", ascending=False).style.format(precision=0)

,antall
elem_type,
,777894227
Illustration,150537814
Advertisement,86441627
Photo,65049189
Table,5391042
Cartoon,3739980
Map,413094
Graph,99077
Photographs,361
